In [3]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [5]:
from torch.utils.data import DataLoader

loaders = {
    'train': DataLoader (train_data,
                         batch_size = 100,
                         shuffle = True,
                         num_workers = 1),
    
    'test': DataLoader(test_data,
                       batch_size = 100,
                       shuffle = True,
                       num_workers = 1)

}

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.conv2_drop = nn.Dropout2d()
        self.fcl = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fcl(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)

        return F.softmax(x)

In [8]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr = 0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders["train"]):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')


def test():
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in loaders["test"]:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim = 1, keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(loaders["test"].dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%\n")
    

In [57]:
for epoch in range(1, 3):
    train(epoch)
    test()

C:\Users\tester\AppData\Local\Temp\ipykernel_15156\756626896.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	1.591875
Train Epoch: 1 [2000/60000 (3%)]	1.618967
Train Epoch: 1 [4000/60000 (7%)]	1.555238
Train Epoch: 1 [6000/60000 (10%)]	1.539209
Train Epoch: 1 [8000/60000 (13%)]	1.562695
Train Epoch: 1 [10000/60000 (17%)]	1.535926
Train Epoch: 1 [12000/60000 (20%)]	1.560713
Train Epoch: 1 [14000/60000 (23%)]	1.582056
Train Epoch: 1 [16000/60000 (27%)]	1.594901
Train Epoch: 1 [18000/60000 (30%)]	1.511962
Train Epoch: 1 [20000/60000 (33%)]	1.579550
Train Epoch: 1 [22000/60000 (37%)]	1.550782
Train Epoch: 1 [24000/60000 (40%)]	1.551860
Train Epoch: 1 [26000/60000 (43%)]	1.585585
Train Epoch: 1 [28000/60000 (47%)]	1.571479
Train Epoch: 1 [30000/60000 (50%)]	1.547976
Train Epoch: 1 [32000/60000 (53%)]	1.572275
Train Epoch: 1 [34000/60000 (57%)]	1.535949
Train Epoch: 1 [36000/60000 (60%)]	1.582677
Train Epoch: 1 [38000/60000 (63%)]	1.541852
Train Epoch: 1 [40000/60000 (67%)]	1.557799
Train Epoch: 1 [42000/60000 (70%)]	1.568006
Train Epoch: 1 [44000/60000 (73%)]	1.553181

In [73]:
import tkinter as tk
from PIL import Image, ImageDraw, ImageOps
import numpy as np
import torch

# Save the trained model
torch.save(model.state_dict(), 'mnist_cnn.pth')



# Initialize the Tkinter window
root = tk.Tk()
root.title("Draw a digit")

# Set up the canvas
canvas = tk.Canvas(root, width=200, height=200, bg="white")
canvas.pack()

# Create an empty image for drawing
image = Image.new("L", (200, 200), color=255)
draw = ImageDraw.Draw(image)

# Function to draw on the canvas
def paint(event):
    x1, y1 = (event.x - 1), (event.y - 1)
    x2, y2 = (event.x + 1), (event.y + 1)
    canvas.create_oval(x1, y1, x2, y2, fill="black", width=5)
    draw.line([x1, y1, x2, y2], fill=0, width=5)

# Bind the paint function to the left mouse button
canvas.bind("<B1-Motion>", paint)

# Function to preprocess the image and make a prediction
def predict():
    # Resize and invert the image (black background)
    resized_image = image.resize((28, 28))
    inverted_image = ImageOps.invert(resized_image)
    
    # Convert the image to a numpy array and normalize it
    image_array = np.array(inverted_image) / 255.0
    image_array = image_array.astype(np.float32)
    image_array = image_array[np.newaxis, np.newaxis, :, :]  # Add batch and channel dimensions

    # Convert the numpy array to a PyTorch tensor
    input_tensor = torch.from_numpy(image_array).to(device)
    
    # Make a prediction using the neural network
    with torch.no_grad():
        output = model(input_tensor)
        prediction = F.softmax(output, dim=1).argmax(dim=1).item()

    # Display the prediction
    result_label.config(text=f"Prediction: {prediction}")

# Load your trained model
model.load_state_dict(torch.load('mnist_cnn.pth'))
model.eval()

# Button to make a prediction
predict_button = tk.Button(root, text="Predict", command=predict)
predict_button.pack()

# Label to display the result
result_label = tk.Label(root, text="Prediction: ")
result_label.pack()

# Start the Tkinter event loop
root.mainloop()

C:\Users\tester\AppData\Local\Temp\ipykernel_15156\756626896.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
